# Setup

In [1]:
# download red wine quality data
! mkdir wine
! wget -P wine https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

# download and unzip Musk v2
! mkdir musk
! wget -P musk https://archive.ics.uci.edu/ml/machine-learning-databases/musk/clean2.data.Z
! uncompress musk/clean2.data.Z

! git clone --branch wip/clustering https://github.com/jsokolowska/anomaly-detection.git
! mv anomaly-detection ad

--2023-05-06 11:17:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘wine/winequality-red.csv’

winequality-red.csv 100%[===================>]  82.23K   318KB/s    in 0.3s    

2023-05-06 11:17:33 (318 KB/s) - ‘wine/winequality-red.csv’ saved [84199/84199]

--2023-05-06 11:17:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/musk/clean2.data.Z
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1470557 (1.4M) [application/x-httpd-php]
Saving to: ‘musk/clean2.data.Z’

clean2.data.Z       100%[==============

In [4]:
# install pyod - not present by default in google colab
! pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184112 sha256=5d7370ecfce40f0d21d067fd99f0e128259d37e56a50036bd667fc27f7e551df
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [7]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

def score( y_true, y_pred, y_proba):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    print(f"F1: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"ROC AUC: {roc}")
    return [precision, recall, f1, roc]

# Parameter Tuning

In [8]:
from ad.src.anomaly_detection import ClusterBasedAnomalyDetection
from sklearn.cluster import Birch
from ad.src.data_loading import load_wine, load_musk

#Birch
n_clusters = 10
birch = Birch(n_clusters=n_clusters)
X, y = load_wine("./wine/winequality-red.csv")

cbad_cblof = ClusterBasedAnomalyDetection(birch, dissimilarity_measure="cblof", measure_args={"n_clusters": n_clusters})
y_scores = cbad_cblof.decision_fun(X)
y_pred = cbad_cblof.detect(X)

print(f"----- Birch clustering with CBLOF -----")
res = score(y, y_pred, y_scores)

/usr/local/lib/python3.10/dist-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"


----- Birch clustering with CBLOF -----
F1: 0.06639004149377593
Precision: 0.05
Recall: 0.09876543209876543
ROC AUC: 0.43262740122643506


/usr/local/lib/python3.10/dist-packages/pyod/models/cblof.py:253: UserWarning: The chosen clustering for CBLOF does not havethe center of clusters. Calculate the centeras the mean of the clusters.
  warnings.warn("The chosen clustering for CBLOF does not have"


In [ ]:
cbad_ldcof = ClusterBasedAnomalyDetection(birch, dissimilarity_measure="ldcof", measure_args={"n_clusters": n_clusters})
y_scores = cbad_ldcof.decision_fun(X)
y_pred = cbad_ldcof.detect(X)

print(f"----- Birch clustering with LDCOF -----")
res = score(y, y_pred, y_scores)

----- Birch clustering with LDCOF -----
F1: 0.07468879668049792
Precision: 0.05625
Recall: 0.1111111111111111
ROC AUC: 0.49668179378324306
